##Capstone project Data Science 

In [166]:
#Correlations without data wrangling
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib as mpl
import matplotlib.path as mpltPath
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


print('Folium installed and imported!')

print("Hello Capstone Project Course!")

Folium installed and imported!
Hello Capstone Project Course!


In [167]:
crash_data = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

print('Data downloaded and read into a dataframe!')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data downloaded and read into a dataframe!


In [168]:
#Change Collisontype and a number of fields into numerical data so correlations can be drawn
#Dummy variables

crash_data.isnull().sum()

SEVERITYCODE           0
X                   5334
Y                   5334
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
EXCEPTRSNCODE     109862
EXCEPTRSNDESC     189035
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        6329
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SDOTCOLNUM         79737
SPEEDING          185340
ST_COLCODE            18
ST_COLDESC          4904
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

In [169]:
# clean up the dataset to remove unnecessary columns (eg. REG) 
#crash_data.drop(['INCKEY','OBJECTID','COLDETKEY','SDOTCOLNUM'], axis=1, inplace=True)


# for sake of consistency, let's also make all column labels of type string
crash_data.columns = list(map(str, crash_data.columns))

#Remove Missing Lightcondition
# simply drop whole row with NaN in "price" column
crash_data.dropna(subset=["X","Y","LIGHTCOND","ADDRTYPE","COLLISIONTYPE","WEATHER","ROADCOND","ST_COLCODE"], axis=0, inplace=True)

# reset index, because we droped two rows
crash_data.reset_index(drop=True, inplace=True)

crash_data.isnull().sum()


SEVERITYCODE           0
X                      0
Y                      0
OBJECTID               0
INCKEY                 0
COLDETKEY              0
REPORTNO               0
STATUS                 0
ADDRTYPE               0
INTKEY            121017
LOCATION               0
EXCEPTRSNCODE     107619
EXCEPTRSNDESC     181151
SEVERITYCODE.1         0
SEVERITYDESC           0
COLLISIONTYPE          0
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDATE                0
INCDTTM                0
JUNCTIONTYPE        4079
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND    155049
UNDERINFL              0
WEATHER                0
ROADCOND               0
LIGHTCOND              0
PEDROWNOTGRNT     179506
SDOTCOLNUM         72520
SPEEDING          175440
ST_COLCODE             0
ST_COLDESC             0
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

In [170]:
for i in crash_data["ADDRTYPE"]: 
    if i == "Intersection":
        Inter = 1
    else:
        Inter = 0
    crash_data["Inter"] = Inter



In [171]:
#After finding the predictive variable, start working on ML (SVM?) For predictive model. 

Preddata = crash_data[["PEDCOUNT", "PEDCYLCOUNT", "Inter", "SEVERITYCODE"]]
Inputdata = crash_data[["PEDCOUNT", "PEDCYLCOUNT", "Inter"]]
y=Preddata[["SEVERITYCODE"]]

X_train, X_test, y_train, y_test = train_test_split(Inputdata, y, test_size=0.2)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [172]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))


Accuracy of logistic regression classifier on test set: 0.75


In [176]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

scores = cross_val_score(logreg, X_train, y_train, cv=10)
print('Cross-Validation Accuracy Scores', scores)

print("Mean AUC Score - Logistic Regression: ", scores.mean())


[[25401   290]
 [ 9021  2118]]
              precision    recall  f1-score   support

           1       0.74      0.99      0.85     25691
           2       0.88      0.19      0.31     11139

   micro avg       0.75      0.75      0.75     36830
   macro avg       0.81      0.59      0.58     36830
weighted avg       0.78      0.75      0.68     36830



/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = colum

Cross-Validation Accuracy Scores [0.7472341  0.74809938 0.74531632 0.74748846 0.74728482 0.74346616
 0.75032245 0.74645306 0.74781074 0.75018668]
Mean AUC Score - Logistic Regression:  0.7473662164335277


In [163]:
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring='roc_auc')


print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was pass

=== Classification Report ===
              precision    recall  f1-score   support

           1       0.74      0.99      0.85     25715
           2       0.89      0.19      0.32     11115

   micro avg       0.75      0.75      0.75     36830
   macro avg       0.81      0.59      0.58     36830
weighted avg       0.78      0.75      0.69     36830



=== All AUC Scores ===
[0.59226874 0.59106378 0.5922001  0.59180917 0.59207221 0.59333649
 0.58742819 0.59346215 0.58845001 0.59219904]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.5914289875630182


In [ ]:
#####X & Y is not predictor, DON"T DO THIS!!!!



#Boundary Box
#47.622153, -122.362590 (LeftTOP)
#47.623137, -122.302495 (RightTOP)
#47.592759, -122.336432 (LeftBOT)
#47.593660, -122.302265 (RightBOT)

def point_in_poly(vertices, y, x):
    from matplotlib.path import Path as mpPath

    path = mpPath(vertices)
    point = (y,x)
    return path.contains_point(point)

vertices = [(47.592759, -122.336432),(47.593660, -122.302265),(47.623137, -122.302495), (47.622153, -122.362590)]
#print(point_in_poly(vertices, 47.602153, -122.312590))

crash_data["Y"].tolist()
crash_data["X"].tolist()
#DataFrame.astype(dtype, copy=True, errors=’raise’)
crash_data["Y"].round(3)
crash_data["X"].round(3)


for column in crash_data[["Y","X"]]:
    point_in_poly(vertices,crash_data["Y"],crash_data["X"])
    